# 1. Information Exchange

In [8]:
import os
import re
import torch
import random
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
LANGUAGE = "java"
CONTEXT_DIR = f'data_{LANGUAGE}/subgraph_contexts'
W2V_MODEL_PATH = f'data_{LANGUAGE}/word2vec.model'
TYPE_TABLE_PATH = f'data_{LANGUAGE}/vuln-char-table-final.csv'
DATASET_DIR = f'data_{LANGUAGE}/dataset_graphs'
EMBED_DIM = 100  # your word2vec dimension
RANDOM_SEED = 42
os.makedirs(DATASET_DIR, exist_ok=True)
w2v = Word2Vec.load(W2V_MODEL_PATH)

In [ ]:
# Step 1: Load node type -> index
def load_type_index():
    import pandas as pd
    df = pd.read_csv(TYPE_TABLE_PATH)
    mapping = {row['node_type']: int(row['index']) for _, row in df.iterrows()}
    return mapping

In [ ]:
type_index_map = load_type_index()

In [ ]:
# Step 2: Tokenizer and vectorizer
def tokenize_code(code):
    code = re.sub(r'[^a-zA-Z0-9_]', ' ', code)
    return code.lower().split()

In [ ]:
def get_code_vector(tokens):
    vecs = []
    for tok in tokens:
        if tok in w2v.wv:
            vecs.append(w2v.wv[tok])
    return np.mean(vecs, axis=0) if vecs else np.zeros(EMBED_DIM)